In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install -U pandas-profiling
!pip install sefr_cut

     |████████████████████████████████| 11.0MB 2.1MB/s 
     |████████████████████████████████| 747kB 28.9MB/s 
     |████████████████████████████████| 1.7MB 3.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 31.6MB/s 
     |████████████████████████████████| 15.3MB 275kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=f92266829d7cadafe1ed7a50b10ee75126df5de33d8823f61d05a24b32993c9a
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 645kB 7.5MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 9.3MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 3.1MB 12.2MB/s 
     |████████████████████████████████| 10.1MB 18.3MB/s 
     |████████████████████████████████| 112kB 47.2MB/s 
     |████████████████████████████████| 296kB 43.8MB/s 
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=4633becdad60a995dbf1126e6fc153245e1e8398d377de3f817255c7a3a53c0f
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d85653789e80e966c3bf6
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=7b02d524a13631a1a4e26eb005c8d5523372f48098b27dd503bf88e6516e9850
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540

In [ ]:
import pandas as pd
import pythainlp
import sefr_cut
import gensim
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

In [ ]:
df = pd.read_csv('CustomerReviews.csv')

In [ ]:
df.head(5)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


Tokenize Words

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n','\n\n', 'ร้าน', '(', ')' , '           ','–', '!', '!!','-','/','+','😆','🤣','"','','%','\u200b','::']
screening_words = stopwords + removed_words

sefr_cut.load_model(engine='ws1000')
def tokenize_with_space(sentence):
  merged = ''
  # words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  words = sefr_cut.tokenize(sentence)
  for word in words[0]:
    if word not in screening_words:
      word = word.rstrip("\n")
      word = word.rstrip("\u200b")
      if word is None or word == ' ' or word == '' or word ==':':
        continue
      else: 
        merged = merged + ',' + word
    elif word is None:
      continue
  return merged[1:]
  
  # return words

loading model.....
Success


In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Review ID         21 non-null     int64  
 1   Restaurant_ID     21 non-null     object 
 2   Restaurant        21 non-null     object 
 3   User              21 non-null     object 
 4   Headline          21 non-null     object 
 5   Review            21 non-null     object 
 6   Rating            20 non-null     float64
 7   Review_tokenized  21 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.4+ KB


In [ ]:
df['Review_tokenized'].iloc[10]


'อัพเดทราคา,ชาบูชิ,ตอน,399,บาท,net,ทาน,1.,15,ชม.,น้ำซุป,เลือก,4,รส,ซูชิลาย,ตา,เทมปุระ,ทาน,เพลิน,ดี,ยืน,ทาน,ไลน์,คาว,บุฟเฟ่ต์,กุ้ง,สด,เบ,เนื้อ,สไลด์,หลัก,อิ่ม,คุ้ม'

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"ลอง,สาขา,ยู,เนี่ยน,มอลล์,รอบ,สาขา,เดอะมอลล์,บา..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,รวมVAT,ทาน,..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,ขึ้นใจ,ชื่อ,ชื่อ,อยู..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้ม,ค่า,สมราคา,บุฟเฟ่,หมู,ผัก,น้ำ,จบ,25..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงิน,สด,ราคา,น้ำ,VAT 7,..."



Create Dictionary

In [ ]:
# documents = df['Review_tokenized'].to_list()
documents = df['Review_tokenized']
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ก็รสชาติ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'จิ้ม', 'ซุป', 'ดี', 'ติม', 'ต้องการ', 'ถาด', 'ถาม', 'น้ำ', 'บริการ', 'บริการรสชาติ', 'บาท', 'ประทับใจ', 'พนักงาน', 'รสชาติ', 'รัก', 'ราคา', 'ร่อย', 'ละกัน', 'สรุป', 'สะอาด', 'สะอ้าน', 'สัน', 'อร่อย', 'อาหาร', 'เคลม', 'เจี้ยว', 'เซ็ท', 'เต้า', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แปลก', 'แฟน', 'โมจิอ', 'ใจ', 'ใส่', 'ไอ', '\u200b ', '\u200bกับ', '\u200bดำ', '\u200bดี', '\u200bพอ', '\u200bภาพ', '\u200bหลากหลาย', '\u200bแต่', '\u200bใหม่', '\u200bไม่', '21', '30', '9', 'CTW', 'Terminal', 'การงดรับ', 'งาน', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ดื่ม', 'ทะยอย', 'ทาน', 'ทำ', 'นาที', 'นึก', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'รายละเอียด', 'ลูกค้า', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่อง', 'เค้า', 'เจอ', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอกาส', 'โอเค', '', '2', '555', '600+', 'กก', 'กรุบ', 'กั้น', 'คน', 'คาว', 'คุโรบู', 'ค่ะ', 'ค่า', 'จาน', 'จืด', 'ฉาก', 'ชอบ', 'ชา', 'ชาบูน้ำดำ', 'ชิ้น', 'ชื่อ', 'ซอยซอส', 'ตก', 'ตะ', 'ตัก', 'ตัว',

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

Topic Modelling

In [ ]:
num_topics = 5
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 100
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 342 ms, sys: 0 ns, total: 342 ms
Wall time: 342 ms


In [ ]:
# pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.057089  0.002533       1        1  32.500727
3      0.027425  0.043027       2        1  27.905696
0      0.023277 -0.039720       3        1  23.317467
2      0.011579 -0.008419       4        1  11.753931
1     -0.005192  0.002579       5        1   4.522179, topic_info=       Term       Freq      Total Category  logprob  loglift
37    เนื้อ  24.000000  24.000000  Default  30.0000  30.0000
3       กิน  22.000000  22.000000  Default  29.0000  29.0000
97       คน  15.000000  15.000000  Default  28.0000  28.0000
16      น้ำ  22.000000  22.000000  Default  27.0000  27.0000
11       ดี  28.000000  28.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
117    นั่ง   0.331516   4.527344   Topic5  -5.1672   0.4820
320    ชั้น   0.315800   4.082627   Topic5  -5.2158   0.5368
22   รสชาติ   0.406314  10.694868   Topic5  -4.9637  -0.1742
105     ชอบ   0.351876   8.893284   Topic5  -5.1076  -0.1336
254    กุ้ง   0.350238  11.094590   Topic5  -5.1123  -0.3594

[344 rows x 6 columns], token_table=      Topic      Freq Term
term                      
90        1  0.140371     
90        2  0.140371     
90        3  0.421114     
90        4  0.280743     
202       2  0.335760    )
...     ...       ...  ...
45        1  0.321652   ไอ
45        2  0.321652   ไอ
45        3  0.160826   ไอ
45        4  0.160826   ไอ
46        1  0.800439   ​ 

[381 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 3, 2])

In [ ]:
model.show_topic(3)

[('ทาน', 0.025617354),
 ('ดี', 0.019625567),
 ('เนื้อ', 0.018004142),
 ('เลือก', 0.017444903),
 ('ราคา', 0.015250688),
 ('อาหาร', 0.015022152),
 ('น้ำ', 0.014040315),
 ('บาท', 0.013645972),
 ('สาขา', 0.011725525),
 ('อร่อย', 0.010808457)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])
df['topic_name'] = df['Review_tokenized'].apply(lambda x: model.show_topic(model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0]))
df['topic_name2'] = df['topic_name'].apply(lambda x:convert_to_topic(x))


In [ ]:
df[['Restaurant', 'Review', 'topics','topic_name2','score']]

,Restaurant,Review,topics,topic_name2,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,4,"[กิน, ดี, อาหาร, ทาน, คน, อร่อย, เนื้อ, สด, น้...",0.998441
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,0,"[ดี, น้ำ, ราคา, เนื้อ, อาหาร, ทาน, หวาน, อร่อย...",0.997820
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,2,"[เนื้อ, น้ำ, ทาน, เลือก, ซุป, อร่อย, , ดี, หวา...",0.999161
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,3,"[ทาน, ดี, เนื้อ, เลือก, ราคา, อาหาร, น้ำ, บาท,...",0.996670
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,3,"[ทาน, ดี, เนื้อ, เลือก, ราคา, อาหาร, น้ำ, บาท,...",0.997436
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,0,"[ดี, น้ำ, ราคา, เนื้อ, อาหาร, ทาน, หวาน, อร่อย...",0.996964
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,0,"[ดี, น้ำ, ราคา, เนื้อ, อาหาร, ทาน, หวาน, อร่อย...",0.998033
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0,"[ดี, น้ำ, ราคา, เนื้อ, อาหาร, ทาน, หวาน, อร่อย...",0.998107
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,0,"[ดี, น้ำ, ราคา, เนื้อ, อาหาร, ทาน, หวาน, อร่อย...",0.997053
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,3,"[ทาน, ดี, เนื้อ, เลือก, ราคา, อาหาร, น้ำ, บาท,...",0.997915


Interpretation

พบว่า Topic Name สามารถแบ่งย่อยได้ 5 เรื่อง 

Topic 0 พูดถึง ดี น้ำ ราคา เนื้อ อาหาร ทาน หวาน อร่อย กิน คุ้ม

Topic 1 พูดถึง กิน ดี เนื้อ คน เลือก ทาน น้ำ ราคา อร่อย บาท

Topic 2 พูดถึง เนื้อ น้ำ ทาน เลือก ซุป อร่อย ดี หวาน กิน

Topic 3 พูดถึง ทาน ดี เนื้อ เลือก ราคา อาหาร น้ำ บาท สาขา อร่อย

Topic 4 พูดถึง กิน ดี อาหาร ทาน คน อร่อย เนื้อ สด น้ำ เลือก



ดังนั้นแต่ละ Topic ควรชื่อว่า

Topic 0 ราคาคุ้ม เนื้อดี อาหารอร่อย 

Topic 1 ราคาดี เนื้ออร่อย

Topic 2 ซุปหวานอร่อย เนื้อดี

Topic 3 เนื้อดี อาหารน้ำคุ้มราคา

Topic 4 เนื้อสด อาหารดี อร่อย เลือกทานได้